In [ ]:
#1. 교차검증(Cross-Validation)


#일반화 성능을 평가하기 위해 훈련 세트와 테스트 세트로 나누어 한번 평가하는 것보다 안정적이고 뛰어난 통계적 평가 방법
#데이터를 여러 번 반복해서 나누고 여러 모델을 학습

#가장 일반적인 교차 검증 방법은 k-겹 교차검증(k-fole cross-validation)으로 k에 보통 5 또는 10을 사용
#데이터를 먼저 k개의 fold(부분집합)으로 나눔
#각각의 폴드를 테스트 세트로 하고 나머지를 훈련 세트로 하여 5번 평가를 반복
#이렇게 하면 k개의 정확도를 얻을 수 있음
#다만, 시간이 더 걸림

In [1]:
from sklearn.model_selection import cross_val_score #folder
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression

import numpy as np
np.set_printoptions(precision=3) #값이 소수점 이하 둘째자리까지만 나오도록 세팅

iris = load_iris()

print("Iris data shape: {}".format(iris.data.shape))
print("Iris labels: {}".format(set(iris.target)))

logreg = LogisticRegression(solver='lbfgs', multi_class='multinomial', max_iter=1000)

# 평가할 모형, 독립변수, 종속변수, k(fold의 수)의 순서로 매개변수를 설정
scores = cross_val_score(logreg, iris.data, iris.target, cv=3) #cv >> data folder의 수
print("Cross-validation scores: {}".format(scores))

Iris data shape: (150, 4)
Iris labels: {0, 1, 2}
Cross-validation scores: [0.98 0.96 0.98]


In [2]:
# k를 5개로 변경해서 평가를 수행
scores = cross_val_score(logreg, iris.data, iris.target, cv=5)
print("Cross-validation scores: {}".format(scores))

Cross-validation scores: [0.967 1.    0.933 0.967 1.   ]


In [5]:
# 얻어진 k개의 정확도에 대해 평균을 계산
print("Average cross-validation score: {:.2f}".format(scores.mean()))

Average cross-validation score: 0.97


In [ ]:
#계층별 k-겹 교차 검증(Stratified K-Fold cross-validation)
#데이터셋을 순서대로 k개의 폴더로 하는 것은 문제 가능성이 있음
#예를 들어 극단적인 경우, 종속변수가 [0,0,0, 1, 1, 1, 2, 2, 2, ]의 형태로 되어 있다면 세 개의 폴더로 했을 때, 학습이 전혀 되지 않는 결과가 발생
#이를 해결하기 위해서는 먼저 random shuffling을 할 수 있고,
#둘째로 계층별 k-겹 교차검증을 할 수 있음
#계층별 k-겹 교차검증은 각 폴드 안의 클래스 비율이 전체 데이터셋의 클래스 비율과 같도록 데이터를 나눔
#분류기에 대해서는 계층별 k-겹 교차검증을 사용하는 것이 보다 안정적임
#클래스간의 inbalanced가 강한 경우 해결(셔플만 잘해줘도 해결이 됨)

In [7]:
#기본적인 사용방법
from sklearn.model_selection import KFold
kfold = KFold(n_splits=5)

In [9]:
print("Cross-validation scores:\n{}".format(
      cross_val_score(logreg, iris.data, iris.target, cv=kfold))) #이렇게만 쓸거면 cv=5써도 ㄱㅊ

Cross-validation scores:
[1.    1.    0.867 0.933 0.833]


In [11]:
kfold = KFold(n_splits=3)
print("Cross-validation scores:\n{}".format(
    cross_val_score(logreg, iris.data, iris.target, cv=kfold)))

Cross-validation scores:
[0. 0. 0.]


In [13]:
# 랜덤 셔플링 등의 조절이 가능
kfold = KFold(n_splits=3, shuffle=True, random_state=0)
print("Cross-validation scores:\n{}".format(
    cross_val_score(logreg, iris.data, iris.target, cv=kfold)))

Cross-validation scores:
[0.98 0.96 0.96]


In [15]:
#LOOCV(Leave-one-out cross-validation)
#폴드 하나에 샘플 하나만 있는 k-겹 교차 검증. 시간이 매우 오래 걸리는 단점이 있음

from sklearn.model_selection import LeaveOneOut
loo = LeaveOneOut()
scores = cross_val_score(logreg, iris.data, iris.target, cv=loo)
print("Number of cv iterations: ", len(scores))
print("Mean accuracy: {:.2f}".format(scores.mean()))
#print(len(iris.data))

Number of cv iterations:  150
Mean accuracy: 0.97


In [17]:
#임의분할 교차검증(Shuffle-split cross-validation) 스토캐스틱
#중복을 허용하는 샘플링, 매번 훈련-테스트 집합을 만들 때마다 전체 데이터셋에서 랜덤하게 샘플링을 수행.

#샘플이 10개일 때, 훈련 세트는 5개, 테스트 세트는 2개로 하여 4번의 세트를 만듦 

from sklearn.model_selection import ShuffleSplit
shuffle_split = ShuffleSplit(test_size=.5, train_size=.5, n_splits=10) #.5 .2 >얼마나 넣을까
scores = cross_val_score(logreg, iris.data, iris.target, cv=shuffle_split)
print("Cross-validation scores:\n{}".format(scores))

Cross-validation scores:
[0.987 0.973 0.96  0.96  0.92  0.96  0.987 0.947 0.973 0.973]


In [ ]:
#매개변수 튜닝을 통한 일반화 성능의 개선방법
#그리드 서치: 매개변수들의 모든 가능한 조합에 대해 테스트를 시도하여 결과가 가장 좋은 매개변수 조합을 찾는 것
#log scale > *10단위로 늘려보는 거

In [23]:
# naive grid search implementation
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target,
                                                    random_state=0)
print("Size of training set: {}   size of test set: {}".format(
      X_train.shape[0], X_test.shape[0]))

best_score = 0
scores = []
parameters= []

for gamma in [0.001, 0.01, 0.1, 1, 10, 100]:
    for C in [0.001, 0.01, 0.1, 1, 10, 100]:
        # for each combination of parameters, train an SVC
        svm = SVC(gamma=gamma, C=C)
        svm.fit(X_train, y_train)
        # evaluate the SVC on the test set
        score = svm.score(X_test, y_test)
        #train.set으로 학습>test.set일반화ㄱㅊ은지 확인
        #이 코드는 답지 확인 함 >튜닝할 때 사용X 
        # >> train셋을 다시 나눠서 체크함. Train, Validation, Test 셋트로 나눔. 
        parameters.append((gamma,C))
        scores.append(score)
        # if we got a better score, store the score and parameters
        if score > best_score:
            best_score = score
            best_parameters = {'C': C, 'gamma': gamma}

print("Best score: {:.2f}".format(best_score))
print("Best parameters: {}".format(best_parameters))

Size of training set: 112   size of test set: 38
Best score: 0.97
Best parameters: {'C': 100, 'gamma': 0.001}


In [25]:
print(scores)

[0.23684210526315788, 0.23684210526315788, 0.23684210526315788, 0.5789473684210527, 0.9210526315789473, 0.9736842105263158, 0.23684210526315788, 0.23684210526315788, 0.5789473684210527, 0.9210526315789473, 0.9736842105263158, 0.9736842105263158, 0.23684210526315788, 0.23684210526315788, 0.9210526315789473, 0.9736842105263158, 0.9736842105263158, 0.9473684210526315, 0.23684210526315788, 0.23684210526315788, 0.9736842105263158, 0.9736842105263158, 0.9736842105263158, 0.9736842105263158, 0.23684210526315788, 0.23684210526315788, 0.2894736842105263, 0.9210526315789473, 0.9210526315789473, 0.9210526315789473, 0.23684210526315788, 0.23684210526315788, 0.23684210526315788, 0.34210526315789475, 0.4473684210526316, 0.4473684210526316]


In [ ]:
# for in zip():

In [27]:
#위 코드는 train.set으로 학습>test.set일반화ㄱㅊ은지 확인
#SO, 이 모형은 답지 확인 함 >튜닝할 때 사용X 
# >> train셋을 다시 나눠서 체크함. Train, Validation, Test 셋트로 나눔. 

from sklearn.svm import SVC
# split data into train+validation set and test set
X_trainval, X_test, y_trainval, y_test = train_test_split(
    iris.data, iris.target, random_state=0)
# split train+validation set into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(
    X_trainval, y_trainval, random_state=1)
print("Size of training set: {}   size of validation set: {}   size of test set:"
      " {}\n".format(X_train.shape[0], X_valid.shape[0], X_test.shape[0]))

best_score = 0

for gamma in [0.001, 0.01, 0.1, 1, 10, 100]:
    for C in [0.001, 0.01, 0.1, 1, 10, 100]:
        # for each combination of parameters train an SVC
        svm = SVC(gamma=gamma, C=C)
        svm.fit(X_train, y_train)
        # evaluate the SVC on the validation set
        score = svm.score(X_valid, y_valid)
        # if we got a better score, store the score and parameters
        if score > best_score:
            best_score = score
            best_parameters = {'C': C, 'gamma': gamma}

# rebuild a model on the combined training and validation set,
# and evaluate it on the test set
svm = SVC(**best_parameters)
svm.fit(X_trainval, y_trainval)
test_score = svm.score(X_test, y_test)
print("Best score on validation set: {:.2f}".format(best_score))
print("Best parameters: ", best_parameters)

# 테스트 스코어가 검증 세트가 없을 때와는 확연히 다른 것을 볼 수 있음
print("Test set score with best parameters: {:.2f}".format(test_score))

Size of training set: 84   size of validation set: 28   size of test set: 38

Best score on validation set: 0.96
Best parameters:  {'C': 10, 'gamma': 0.001}
Test set score with best parameters: 0.92
